In [1]:
#interface graphique 
import tkinter as tk
import pprint
import random
import time
#import de notebook
import nbimporter
from ia import Ia_dumb
Ia_dumb()
import sys

Importing Jupyter notebook from ia.ipynb
running


<h2><b>GAME</b></h2>

In [2]:
#function pour dessiner 
def pack_util(p) : 
    return p.pack()

In [3]:
score = 0

#classe principale 
class Snake_game :
    def __init__(self, tinker, size = 500, play_btn = True):
        #initialisation des variables
        self.game_size = size
        self.pixel_size = 20
        self.tinker = tinker
        self.commands = {
            "z" : "top",
            "q" : "left",
            "d" : "right",
            "s" : "bottom"
        }
        self.reverse_commands = {
            "top" : "bottom",
            "right" : "left",
            "left" : "right",
            "bottom" : "top",
        }
        self.direction = ""
        self.commands_history = [self.direction]
        
        #initialisation fenêtre tkinter
        self.tinker.title("Snake")
        self.tinker.resizable(False, False)
        self.tinker.tk.call("tk", "scaling", 4.0)
        
        #initialisation de la matrice / creation d'une ligne vide
        self.grid_pos = [[] for i in range(0, int(self.game_size / self.pixel_size))]
        self.create_grid()
        
        #booleen pour afficher le bouton play 
        if play_btn : 
            #bouton jouer
            self.play_button = tk.Button(self.tinker, text = "PLAY", command = self.play, width = 8, height = 2)
            pack_util(self.play_button)
            
    def create_canvas(self) :
        #canvas
        canvas = tk.Canvas(self.tinker, width = self.game_size, height = self.game_size, background = "black", highlightthickness = 0)
        return canvas
          
    def create_grid(self) :
        
        #initialisation d'un dictionnaire
        pos_dict = {}             
        #remplir chaque ligne avec une liste de tuple de coordonnées possibles  
        for x in range(0, self.game_size + 1, self.pixel_size) :
            pos_dict[x] = []
            for y in range(0, self.game_size + 1, self.pixel_size) :
                pos_dict[x].append((x, y))

        #création de matrice      
        for x in list(pos_dict.keys())[:-1]:
            for i in range(len(pos_dict[x])-1):
                self.grid_pos[i].append([pos_dict[x][i],pos_dict[x+self.pixel_size][i+1]])
        
    #fonction d'écoute de touche    
    def set_key_event(self, event, canvas, apple) :
        #vérifier la touche 
        if event.char in self.commands :
            #définir la direction
            self.direction = self.commands[event.char]           
            #on vérifie si l'utilisateur n'appuie pas deux fois sur la même touche
            if (self.commands_history[-1] != self.direction) and (self.commands_history[-1] != self.reverse_commands[self.direction]) : 
                    self.commands_history.append(self.direction)
                    self.snake.move(self.direction, canvas, self.commands_history, apple)
    
    def play(self) :
        print("Playing")
        #retirer le bouton play 
        self.play_button.destroy()
        
        #initialisation canvas
        self.canvas = self.create_canvas()
        
        #on vérifie si il y a plus de 1 élément fils sur la fenêtre
        if len(self.tinker.winfo_children()) == 1 :
            #initialisation de la pomme
            self.apple = Apple(self.tinker, self.game_size, False)
            self.apple.create_apple(self.canvas)
            
            #initialisation du snake
            self.snake = Snake(self.tinker, self.game_size, False)
            self.snake.create_snake(self.canvas)
            
            #écouter touche du clavier
            self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas, self.apple))
            
            pack_util(self.canvas)
    
    def game_over(self, canvas):
        canvas.delete(tk.ALL)
        global score
        canvas.create_text(
            canvas.winfo_width() / 2,
            canvas.winfo_height() / 2,
            text=f"GAME OVER ! Vous avez remporté {score} point(s).",
            fill="#FFFFFF",
            font=("Helvetica", 5)
        )

In [4]:
#TODO set Apple 
class Apple (Snake_game):
    def __init__(self, tinker, game_size, play_btn):
        super().__init__(tinker, game_size, False)
        self.init_pos = int((self.game_size/self.pixel_size)/2)

    def create_apple(self, canvas):
        #print("apple", canvas)
        #création de la pomme
        self.apple = canvas.create_rectangle(
                self.grid_pos[self.init_pos][self.init_pos][0][1],
                self.grid_pos[self.init_pos][self.init_pos][0][0],
                self.grid_pos[self.init_pos][self.init_pos][1][1],
                self.grid_pos[self.init_pos][self.init_pos][1][0],
                outline="#000000",
                fill="#ff8f26",
                tags=('apple')
            )
    def update_apple(self, canvas, apple_id) :
        x = random.randint(5, int((self.game_size/self.pixel_size)/1.1))
        y = random.randint(5, int((self.game_size/self.pixel_size)/1.1))
        print(x,y,"appllllllle")
        #dessin du snake
        if x < len(self.grid_pos) :
            if y < len(self.grid_pos[x]):
                canvas.coords(
                    apple_id,
                    self.grid_pos[x][y][0][1],
                    self.grid_pos[x][y][0][0],
                    self.grid_pos[x][y][1][1],
                    self.grid_pos[x][y][1][0],
                )
            else:
                print(f"index error y update_apple {x, y}")
        else:
            print(f"index error x update_apple {x, y}")
        

In [5]:
class Snake (Snake_game) :
    def __init__(self, tinker, game_size, play_btn):
        super().__init__(tinker, game_size, False)
        #initialisation position snake
        self.snake_head = (7, 5)
        self.snake_positions = [self.snake_head, (6,5), (5,5)]
        self.direction = "none"
        
    def create_snake(self, canvas):
        #print("snake", canvas)
        for i, position in enumerate(self.snake_positions) :
            self.snake_part = canvas.create_rectangle(
                self.grid_pos[position[0]][position[1]][0][1],
                self.grid_pos[position[0]][position[1]][0][0],
                self.grid_pos[position[0]][position[1]][1][1],
                self.grid_pos[position[0]][position[1]][1][0],
                outline="#000000",
                fill="#1EC81C",
                tags=('snake_head') if i == 0 else ('snake')
            )
    def add_snake_part(self, canvas, x, y):
        if x < len(self.grid_pos) :
            if y < len(self.grid_pos[x]):
                self.snake_part = canvas.create_rectangle(
                        self.grid_pos[x][y][0][1],
                        self.grid_pos[x][y][0][0],
                        self.grid_pos[x][y][1][1],
                        self.grid_pos[x][y][1][0],
                        outline="#000000",
                        fill="#1EC81C",
                        tags=('snake')
                    )
            else:
                print(f"index error y add_snake_part {x, y}")
        else:
            print(f"index error x add_snake_part {x, y}")
    
    def update_snake(self, canvas, snake_id, x, y) :
        if x < len(self.grid_pos) :
            if y < len(self.grid_pos[x]):
                #dessin du snake
                canvas.coords(
                    snake_id,
                    self.grid_pos[x][y][0][1],
                    self.grid_pos[x][y][0][0],
                    self.grid_pos[x][y][1][1],
                    self.grid_pos[x][y][1][0],
                )
            else:
                print(f"index error y update_snake {x, y}")
        else:
            print(f"index error x update_snake {x, y}")
    
                
            
    def move(self, direction, canvas, commands_history, apple) :
        
        
        
        #controle de la boucle
        movement = True
        
        #changement de direction
        if len(commands_history) > 2 : 
            self.direction_is_diff = True if (direction == "right") or (direction == "left") else False
        else :
            self.direction_is_diff = False
        print(len(self.grid_pos))
        
        #boucle de mouvement
        while movement :
            if self.is_touched():
                self.game_over(canvas)
                break
            
            #id tête du snake
            self.snake_head_ids = canvas.find_withtag("snake_head")

            #tout les ids du corps du snake
            self.snake_parts_ids = canvas.find_withtag("snake")

            #id de tout le snake
            self.snake_ids =  self.snake_parts_ids + self.snake_head_ids
            
            #id de la pomme
            self.apple_id = canvas.find_withtag("apple")
            
            for i, snake_id in enumerate(self.snake_ids):
                #coordonné d'une partie du snake par id
                coords = canvas.coords(snake_id)
                
                apple_coords = canvas.coords(self.apple_id)                

                x = self.snake_positions[i][0]
                y = self.snake_positions[i][1]

                head_x = self.snake_head[0]
                head_y = self.snake_head[1]
                
                #mouvement à droite
                if direction == "right" :
                    y = head_y
                    #on vérifie que ça ne touche pas les bords 
                    if self.direction_is_diff:
                        x += i+1
                        time.sleep(0.01*i)
                        canvas.update()
                        if i == len(self.snake_ids) - 1 :
                            self.direction_is_diff = False
                            #print("changed is diff", self.direction_is_diff)
                    else :
                        x += 1
                    self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head     
                    self.snake_positions[i] = (x,y)
                    self.update_snake(canvas, snake_id, x, y)
                            
                
                if direction == "left" :
                    y = head_y
                    #print("self.direction_is_diff", self.direction_is_diff)
                    if self.direction_is_diff :
                        x -= i+1
                        time.sleep(0.01*i)
                        canvas.update()
                        if i == len( self.snake_ids) - 1 :
                            self.direction_is_diff = False
                            #print("changed is diff", self.direction_is_diff)
                    else :
                        x -= 1
                    self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head 
                    self.snake_positions[i] = (x,y)
                    self.update_snake(canvas, snake_id, x, y)
                        
                if direction == "bottom" :
                    x = head_x
                    if self.direction_is_diff == False:
                        y += i+1
                        time.sleep(0.01*i)
                        canvas.update()
                        if i == len (self.snake_ids) - 1 :
                            self.direction_is_diff = True
                            #print("changed is diff", self.direction_is_diff)
                    else :
                        y += 1
                    self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head 
                    self.snake_positions[i] = (x,y)
                    self.update_snake(canvas, snake_id, x, y)
                
                if direction == "top" :
                    x = head_x
                    if self.direction_is_diff == False:
                        y -= i+1
                        time.sleep(0.01*i)
                        canvas.update()
                        self.update_snake(canvas, snake_id, x, y)
                        if i == len (self.snake_ids) - 1 :
                            self.direction_is_diff = True
                            #print("changed is diff", self.direction_is_diff)
                    else :
                        y -= 1
                    self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head 
                    self.snake_positions[i] = (x,y)
                    self.update_snake(canvas, snake_id, x, y)

                #print("snake_head", self.snake_head_ids[0])
                #print(f"Snake_part-{snake_id} : {x, y}, direction : {direction}, self.direction_is_diff : {self.direction_is_diff}")
                
                if coords == apple_coords :
                    last_x = self.snake_positions[-1][0]
                    last_y = self.snake_positions[-1][1]
                    apple.update_apple(canvas, self.apple_id)
                    pos = (last_x, last_y + 1) if (direction == "right") or (direction == "left") else (last_x + 1, last_y)
                    self.snake_positions.append((pos[0], pos[1]))
                    self.add_snake_part(canvas, pos[0], pos[1])
                    global score
                    score += 1
                    
            time.sleep(0.12)
            #update du canvas
            canvas.update()
            
        print(direction)
    
    def is_touched(self):
        #les coordonnées de la tête
        x, y = self.snake_positions[0]
        #vérifier si le serpent touche les bords ou lui-même
        if x in (0, 25) or y in (0, 25) or (x, y) in self.snake_positions[1:]:
            return True

In [6]:
root = tk.Tk()
snake_game = Snake_game(root)
root.mainloop()

Playing
25
index error x update_snake (25, 5)
right
